In [1]:
import pycisTopic
pycisTopic.__version__

'0.1.dev300+g7494158'

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
import pickle
import pandas as pd

In [4]:
import copy

In [5]:
import os
wdir = '/lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/'
os.chdir( wdir )

In [6]:
# create output directory:
f_final_dir = os.path.join(wdir, 'downstream_analysis')
if not os.path.exists(f_final_dir):
    os.makedirs(f_final_dir)

In [7]:
# display names of each sample. NOTE: the samples will be dislpayed in the order of this dict!
from collections import OrderedDict
alias_dict = OrderedDict({
    "Broad_1": "BioRad ATAC 1",
    "Broad_2": "BioRad ATAC 2",
    "Stanford_1": "10x ATAC A1",
    "Stanford_2": "10x ATAC A2",
    "VIB_1": "10x ATAC B1",
    "VIB_2": "10x ATAC B2",
    "CNAG_1": "10x ATAC C1",
    "CNAG_2": "10x ATAC C2",
    "Broad_mito_1": "10x mtATAC 1",
    "Broad_mito_2": "10x mtATAC 2",
    "Sanger_1": "10x Multiome 1",
    "Sanger_2": "10x Multiome 2",
    "VIB_Hydrop_1": "Hydrop ATAC 1",
    "VIB_Hydrop_2": "Hydrop ATAC 2",
    "s3atac": "s3 ATAC",
})

color_dict = OrderedDict({
    "Broad_mito_1": "#9467bd",
    "Broad_mito_2": "#c5b0d5",
    "CNAG_1": "#d62728",
    "CNAG_2": "#ff9896",
    "Broad_1": "#1f77b4",
    "Broad_2": "#aec7e8",
    "Sanger_1": "#8c564b",
    "Sanger_2": "#c49c94",
    "Stanford_1": "#ff7f0e",
    "Stanford_2": "#ffbb78",
    "VIB_1": "#2ca02c",
    "VIB_2": "#98df8a",
    "VIB_Hydrop_1": "#e377c2",
    "VIB_Hydrop_2": "#f7b6d2",
    "s3atac": "#7f7f7f",
})

tech_dict = OrderedDict({
    "Broad_1": "BioRad ATAC",
    "Broad_2": "BioRad ATAC",
    "Stanford_1": "10x ATAC A",
    "Stanford_2": "10x ATAC A",
    "VIB_1": "10x ATAC B",
    "VIB_2": "10x ATAC B",
    "CNAG_1": "10x ATAC C",
    "CNAG_2": "10x ATAC C",
    "Broad_mito_1": "10x mtATAC",
    "Broad_mito_2": "10x mtATAC",
    "Sanger_1": "10x Multiome",
    "Sanger_2": "10x Multiome",
    "VIB_Hydrop_1": "Hydrop ATAC",
    "VIB_Hydrop_2": "Hydrop ATAC",
    "s3atac": "s3 ATAC",
})

## Load the cisTopic objects

In [8]:
import glob
f_cto_dir = 'pycistopic_consensus_peaks/cistopic_objs__consensus'

glob.glob(f_cto_dir + '/*__cistopic_obj_metadata_annotated_dimreduc_df.pkl')

['pycistopic_consensus_peaks/cistopic_objs__consensus/s3atac__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/CNAG_2__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/VIB_2__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/VIB_1__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/Sanger_2__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/Broad_2__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/VIB_Hydrop_1__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/Broad_mito_2__cistopic_obj_metadata_annotated_dimreduc_df.pkl',
 'pycistopic_consensus_peaks/cistopic_objs__consensus/Broad_mito_1__cistopic_obj_metadata_a

In [9]:
cistopic_obj_dict = {}
for key in alias_dict.keys():
    f_cto = os.path.join(wdir, f_cto_dir, key + '__cistopic_obj_metadata_annotated_dimreduc_df.pkl')
    if(os.path.isfile(f_cto)):
        with open(f_cto, 'rb') as f:
            cistopic_obj_dict[key] = pickle.load(f)
        print(f"Loaded filtered cistopic object {key}")
    else:
        print(f"file {f_cto} doesn't exist")

Loaded filtered cistopic object Broad_1
Loaded filtered cistopic object Broad_2
Loaded filtered cistopic object Stanford_1
Loaded filtered cistopic object Stanford_2
Loaded filtered cistopic object VIB_1
Loaded filtered cistopic object VIB_2
Loaded filtered cistopic object CNAG_1
Loaded filtered cistopic object CNAG_2
Loaded filtered cistopic object Broad_mito_1
Loaded filtered cistopic object Broad_mito_2
Loaded filtered cistopic object Sanger_1
Loaded filtered cistopic object Sanger_2
Loaded filtered cistopic object VIB_Hydrop_1
Loaded filtered cistopic object VIB_Hydrop_2
Loaded filtered cistopic object s3atac


In [19]:
cistopic_obj_dict['Broad_1'].cell_data

,sample_id,cisTopic_nr_frag,cisTopic_log_nr_frag,Total_nr_frag,pycisTopic_leiden_10_0.6,Log_unique_nr_frag,Dupl_rate,cisTopic_nr_acc,TSS_enrichment,seurat_cell_type,...,FRIP,barcode,fmx_sample,Unique_nr_frag,pycisTopic_leiden_10_0.8,pycisTopic_leiden_10_1.0,UMAP_1,UMAP_2,tSNE_1,tSNE_2
CCTTAGGGCGACTCAGCGAAT_AGCTGAGCGTACAAACAGCTT-Broad_1,Broad_1,14292,4.155093,18048,2,4.256429,0.697739,10940,32.221465,Natural killer cell,...,0.889129,CCTTAGGGCGACTCAGCGAAT_AGCTGAGCGTACAAACAGCTT,sampleA,18048,2,2,5.325886,1.055166,-36.112318,42.185708
ATTGCCTAATGATTAGTAAGC_ATGAGCTAACCACAGTCGGTT_CACGGACATAGGCAGAATCAA-Broad_1,Broad_1,6759,3.829882,9147,1,3.961279,0.672702,5768,33.876631,CD4+ T cell,...,0.831311,ATTGCCTAATGATTAGTAAGC_ATGAGCTAACCACAGTCGGTT_CA...,sampleA,9147,1,1,13.743738,13.992117,2.050739,-41.636173
ACTACGACGTGATAGGTTCAC_CGGCTGCCCAAGCTTATGTGC_AACGTAAAATGGCCGTTGATG-Broad_1,Broad_1,7065,3.849112,9222,0,3.964825,0.692179,5960,25.272116,CD4+ T cell,...,0.846996,ACTACGACGTGATAGGTTCAC_CGGCTGCCCAAGCTTATGTGC_AA...,sampleB,9222,0,0,11.444382,12.942389,-1.088673,-21.098056
CTTGTCCAACGGTGTGGACCA_CCGAACCATAGGCACCAGTCA_AGCGAATTTTGGTCTTGGACT-Broad_1,Broad_1,4513,3.654465,6244,9,3.795463,0.667519,3962,35.924081,CD4+ T cell,...,0.820468,CTTGTCCAACGGTGTGGACCA_CCGAACCATAGGCACCAGTCA_AG...,sampleB,6244,3,8,12.773808,11.500572,-15.946676,-23.092541
TTAAGCGTGGACCAAATTCCA_AACGGTGGCTTTATCATGTAT-Broad_1,Broad_1,5871,3.768712,8088,1,3.907841,0.652428,5076,32.131229,CD4+ T cell,...,0.801187,TTAAGCGTGGACCAAATTCCA_AACGGTGGCTTTATCATGTAT,sampleA,8088,1,1,12.408016,13.772733,7.745348,-24.685312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GCTAATTTAAGGTACACATGA_CACGCCACATCAGTCCTATGT_GTCCTTCTGTCGCTTAGTACC_TGGCCTTTCCTGGCAATTGGT-Broad_1,Broad_1,2688,3.429429,3280,2,3.515874,0.656293,2478,42.434688,Cytotoxic T cell,...,0.923476,GCTAATTTAAGGTACACATGA_CACGCCACATCAGTCCTATGT_GT...,sampleB,3280,2,12,4.734689,3.628466,-39.278439,18.369763
ATTATTCCGCCTAATCTGAAC_TACCGAACGTTTGAGAGCGTG_TACGCATAATTGGTTGAGATT_GTGCATTTCATACACCACTTG_GAGGACAGACAATAAGCTGAG-Broad_1,Broad_1,3178,3.502154,4187,3,3.621903,0.661739,2884,42.318125,CD4+ T cell,...,0.856699,ATTATTCCGCCTAATCTGAAC_TACCGAACGTTTGAGAGCGTG_TA...,sampleB,4187,3,8,13.708868,10.603662,-26.271765,-28.124069
TTGTAAGCGACACTTGTCGCT_TTGTAAGCAGAATTGAACCGT_TTGTAAGGTCCTTCATATAAC_CGGCTGCGCGTAGACGATTAC-Broad_1,Broad_1,2466,3.391993,3081,6,3.488692,0.665908,2282,40.103818,Cytotoxic T cell,...,0.896138,TTGTAAGCGACACTTGTCGCT_TTGTAAGCAGAATTGAACCGT_TT...,sampleB,3081,7,6,6.157417,9.925058,-18.835261,7.778407
GCGACTCGGAAGTTGTCGGTT_ACGTTGGTAAGGTAAGCTGAG_AACCACAAACGTAATCGACAG_CCTTAGGCCTTAGGGGTTCAC-Broad_1,Broad_1,3587,3.554731,4605,9,3.663230,0.657366,3209,39.794409,CD4+ T cell,...,0.881216,GCGACTCGGAAGTTGTCGGTT_ACGTTGGTAAGGTAAGCTGAG_AA...,sampleB,4605,3,8,12.611784,10.811452,-21.424586,-21.071385


#### Save/load
Only necessary if you have not calculated accessibility yet.

In [22]:
f_out = os.path.join(f_final_dir, 'region_bin_topics.pkl')
if os.path.isfile(f_out):
    with open(f_out, 'rb') as f:
        region_bin_topics_dict = pickle.load(f)
    print(f"Loaded {f_out}")

Loaded /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/region_bin_topics.pkl


In [24]:
region_bin_topics_dict.keys()

dict_keys(['VIB_Hydrop_1', 'VIB_Hydrop_2', 'Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1', 'CNAG_2', 's3atac', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2'])

In [29]:
f_out = os.path.join(f_final_dir, 'binarized_cell_topic.pkl')
if os.path.isfile(f_out):
    with open(f_out, 'rb') as f:
        binarized_cell_topic_dict = pickle.load(f)
    print(f"Loaded {f_out}")

Loaded /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/binarized_cell_topic.pkl


In [30]:
binarized_cell_topic_dict.keys()

dict_keys(['VIB_Hydrop_1', 'VIB_Hydrop_2', 'Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1', 'CNAG_2', 's3atac', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2'])

In [33]:
f_out = os.path.join(f_final_dir, 'imputed_acc_obj.pkl')
if os.path.isfile(f_out):
    with open(f_out, 'rb') as f:
        imputed_acc_obj_dict = pickle.load(f)
    print(f"Loaded {f_out}")

Loaded /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/imputed_acc_obj.pkl


In [35]:
imputed_acc_obj_dict.keys()

dict_keys(['VIB_Hydrop_1', 'VIB_Hydrop_2', 'Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1', 'CNAG_2', 's3atac', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2'])

In [36]:
f_out = os.path.join(f_final_dir, 'markers_dict.pkl')
if os.path.isfile(f_out):
    with open(f_out, 'rb') as f:
        markers_dict_dict = pickle.load(f)
    print(f"Loaded {f_out}")

Loaded /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/markers_dict.pkl


In [38]:
markers_dict_dict.keys()

dict_keys(['VIB_Hydrop_1', 'VIB_Hydrop_2', 'Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1', 'CNAG_2', 's3atac', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2'])

## Infer or load gene activity

In [39]:
import pyranges as pr
import requests
import pybiomart as pbm

In [40]:
from pycisTopic.gene_activity import get_gene_activity
from pycisTopic.diff_features import find_diff_features

In [41]:
f_out = os.path.join(f_final_dir, 'gene_act_dict.pkl')
if os.path.isfile(f_out):
    print(f"Loading {f_out}")
    with open(f_out, 'rb') as f:
        gene_act_dict = pickle.load(f)
else:
    from pycisTopic.gene_activity import get_gene_activity
    from pycisTopic.diff_features import find_diff_features
    # For human
    dataset = pbm.Dataset(name='hsapiens_gene_ensembl',  host='http://www.ensembl.org')
    annot = dataset.query(attributes=['chromosome_name', 'start_position', 'end_position', 'strand', 'external_gene_name', 'transcription_start_site', 'transcript_biotype'])
    annot['Chromosome/scaffold name'] = 'chr' + annot['Chromosome/scaffold name'].astype(str)
    annot.columns=['Chromosome', 'Start', 'End', 'Strand', 'Gene','Transcription_Start_Site', 'Transcript_type']
    annot = annot[annot.Transcript_type == 'protein_coding']
    annot.Strand[annot.Strand == 1] = '+'
    annot.Strand[annot.Strand == -1] = '-'
    pr_annotation = pr.PyRanges(annot.dropna(axis = 0))
    pr_annotation

    # get chromosome sizes (hg38)
    target_url = 'http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes'
    chromsizes = pd.read_csv(target_url, sep='\t', header=None)
    chromsizes.columns = ['Chromosome', 'End']
    chromsizes['Start'] = [0]*chromsizes.shape[0]
    chromsizes = chromsizes.loc[:,['Chromosome', 'Start', 'End']]
    chromsizes = pr.PyRanges(chromsizes)
    chromsizes

    gene_act_dict = {}
    for key in cistopic_obj_dict.keys():
        print(key)
        gene_act, weights = get_gene_activity(
            imputed_acc_obj_dict[key], # Region-cell probabilities
            pr_annotation, # Gene annotation
            chromsizes, # Chromosome size
            use_gene_boundaries=True, # Whether to use the whole search space or stop when encountering another gene
            upstream=[1000, 100000], # Search space upstream. The minimum means that even if there is a gene right next to it 
                                     #these bp will be taken (1kbp here)
            downstream=[1000,100000], # Search space downstream
            distance_weight=True, # Whether to add a distance weight (an exponential function, the weight will decrease with distance)
            decay_rate=1, # Exponent for the distance exponential funciton (the higher the faster will be the decrease)
            extend_gene_body_upstream=10000, # Number of bp upstream immune to the distance weight (their value will be maximum for 
                                             #this weight)
            extend_gene_body_downstream=500, # Number of bp downstream immune to the distance weight
            gene_size_weight=False, # Whether to add a weights based on the length of the gene
            gene_size_scale_factor='median', # Dividend to calculate the gene size weigth. Default is the median value of all genes
                                             #in the genome
            remove_promoters=False, # Whether to remove promoters when computing gene activity scores
            average_scores=True, # Whether to divide by the total number of region assigned to a gene when calculating the gene 
                                 # activity score
            scale_factor=1, # Value to multiply for the final gene activity matrix
            extend_tss=[10,10], # Space to consider a promoter
            gini_weight = True, # Whether to add a gini index weigth. The more unique the region is, the higher this weight will be
            return_weights= True, # Whether to return the final weights
            project='Gene_activity') # Project name for the gene activity object
        gene_act_dict[key] = copy.copy(gene_act)

    with open(f_out, 'wb') as f:
        pickle.dump(gene_act_dict, f)
    print(f'saved {f_out}')

Broad_1
2022-01-10 18:06:52,166 cisTopic     INFO     Calculating gene boundaries


join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.


2022-01-10 18:07:02,509 cisTopic     INFO     Calculating distances
2022-01-10 18:07:27,897 cisTopic     INFO     Calculating distance weigths
2022-01-10 18:07:28,598 cisTopic     INFO     Distance weights done
2022-01-10 18:07:28,599 cisTopic     INFO     Calculating gini weights
2022-01-10 18:08:08,618 cisTopic     INFO     Getting gene activity scores
2022-01-10 18:11:21,553 cisTopic     INFO     Creating imputed features object
Broad_2
2022-01-10 18:11:27,851 cisTopic     INFO     Calculating gene boundaries


join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.


2022-01-10 18:11:37,558 cisTopic     INFO     Calculating distances
2022-01-10 18:12:02,730 cisTopic     INFO     Calculating distance weigths
2022-01-10 18:12:03,430 cisTopic     INFO     Distance weights done
2022-01-10 18:12:03,431 cisTopic     INFO     Calculating gini weights
2022-01-10 18:12:41,140 cisTopic     INFO     Getting gene activity scores
2022-01-10 18:15:46,950 cisTopic     INFO     Creating imputed features object
Stanford_1
2022-01-10 18:15:52,296 cisTopic     INFO     Calculating gene boundaries


...

2022-01-10 19:14:21,680 cisTopic     INFO     Calculating distance weigths
2022-01-10 19:14:22,309 cisTopic     INFO     Distance weights done
2022-01-10 19:14:22,310 cisTopic     INFO     Calculating gini weights
2022-01-10 19:14:42,822 cisTopic     INFO     Getting gene activity scores
2022-01-10 19:16:51,688 cisTopic     INFO     Creating imputed features object
VIB_Hydrop_2
2022-01-10 19:16:56,967 cisTopic     INFO     Calculating gene boundaries
2022-01-10 19:17:07,496 cisTopic     INFO     Calculating distances


join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.


2022-01-10 19:17:27,510 cisTopic     INFO     Calculating distance weigths
2022-01-10 19:17:28,121 cisTopic     INFO     Distance weights done
2022-01-10 19:17:28,122 cisTopic     INFO     Calculating gini weights
2022-01-10 19:17:52,204 cisTopic     INFO     Getting gene activity scores
2022-01-10 19:20:14,608 cisTopic     INFO     Creating imputed features object
s3atac
2022-01-10 19:20:21,662 cisTopic     INFO     Calculating gene boundaries


join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.


2022-01-10 19:20:31,932 cisTopic     INFO     Calculating distances
2022-01-10 19:21:02,488 cisTopic     INFO     Calculating distance weigths
2022-01-10 19:21:03,234 cisTopic     INFO     Distance weights done
2022-01-10 19:21:03,235 cisTopic     INFO     Calculating gini weights
2022-01-10 19:21:48,974 cisTopic     INFO     Getting gene activity scores
2022-01-10 19:25:27,317 cisTopic     INFO     Creating imputed features object
saved /lustre1/project/stg_00002/lcb/fderop/data/20211215_hca_benchmark_github/4_downsampled_data_individual_samples/downstream_analysis/gene_act_dict.pkl


In [42]:
gene_act_dict

{'Broad_1': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2afac12b52e0>,
 'Broad_2': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2afaaefb5970>,
 'Stanford_1': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2afab5f3aee0>,
 'Stanford_2': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2afabfcdb9a0>,
 'VIB_1': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2afac21282e0>,
 'VIB_2': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2afabc9fc190>,
 'CNAG_1': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2afabd2cd0d0>,
 'CNAG_2': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2afab376adc0>,
 'Broad_mito_1': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2afac18ec340>,
 'Broad_mito_2': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2afaa7531820>,
 'Sanger_1': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2afab0e0f3a0>,
 'Sanger_2': <pycisTopic.diff_features.CistopicImputedFeatures at 0x2afabfcf28e0>,
 'VI

In [43]:
for key in gene_act_dict.keys():
    f_out = os.path.join(f_final_dir, key + '__libds__gene_act.pkl')
    if os.path.isfile(f_out):
        print(f"Saving {f_out}")
        with open(f_out, 'wb') as f:
            pickle.dump(gene_act_dict[key], f)

## Export to loom

In [16]:
f_gene_loom_dir = os.path.join(f_final_dir, 'gene_act_loom')
if not os.path.exists(f_gene_loom_dir):
    os.makedirs(f_gene_loom_dir)

In [17]:
from pycisTopic.loom import (
    export_region_accessibility_to_loom,
    export_gene_activity_to_loom
)

### Gene activity

In [18]:
from ctxcore.genesig import Regulon

# generate a dummy regulon (required for export_gene_activity_to_loom):
phreg = Regulon(
        name='placeholder regulon',
        gene2weight={'phreg': 1.0},
        transcription_factor="phreg",
        gene2occurrence={"phreg": 1},
    )

In [19]:
for key in cistopic_obj_dict.keys():
    print(key)
    f_out = os.path.join(f_gene_loom_dir, key + '__libDS_gene_activity.loom')
    if os.path.exists(f_out):
        print(f"Skipping {f_out}: already exists.")
        continue
        
    export_gene_activity_to_loom(
        gene_activity_matrix = gene_act_dict[key],
        cistopic_obj = cistopic_obj_dict[key], 
        regulons = [phreg],
        # selected_cells = [ x.split('-')[0] + '-' + x.split('-')[1]  for x in cistopic_obj_dict[key].cell_names ], # this leaves a cell barcode of the format type 'TGCATGTCGCCGTTCCAAGA-21'
        # selected_cells = cistopic_obj_dict[key].projections['cell']['UMAP'].index.tolist(), # cflerin original 
        out_fname = f_out,t
        cluster_annotation = ['consensus_cell_type', 'SCREEN_fmx_sample'],
        cluster_markers = {'consensus_cell_type': markers_dict_dict[key], 'SCREEN_fmx_sample':{}},
        tree_structure = ('scATAC-seq_Benchmark', 'ATAC_library_downsampled', 'Gene_activity'),
        title = 'Gene activity from library-downsampled and sample-merged dataset',
        nomenclature = "hg38"
    )

VIB_Hydrop_1
2021-11-26 13:58:48,686 cisTopic     INFO     Creating minimal loom
2021-11-26 13:58:53,254 ctxcore.recovery WARNING  Less than 80% of the genes in placeholder regulon are present in the expression matrix.
2021-11-26 13:59:03,727 cisTopic     INFO     Adding annotations
2021-11-26 13:59:05,305 cisTopic     INFO     Adding clusterings
2021-11-26 13:59:05,323 cisTopic     INFO     Adding markers
No markers for  sampleA
No markers for  sampleB
2021-11-26 13:59:05,568 cisTopic     INFO     Exporting
VIB_Hydrop_2
2021-11-26 13:59:16,087 cisTopic     INFO     Creating minimal loom
2021-11-26 13:59:21,274 ctxcore.recovery WARNING  Less than 80% of the genes in placeholder regulon are present in the expression matrix.
2021-11-26 13:59:30,968 cisTopic     INFO     Adding annotations
2021-11-26 13:59:32,779 cisTopic     INFO     Adding clusterings
2021-11-26 13:59:32,799 cisTopic     INFO     Adding markers
No markers for  sampleA
No markers for  sampleB
2021-11-26 13:59:33,038 cisT

# Caution:
The export loom function contains a bug which adds the barcodes as a clustering, causing scope to crash on loading. This redundant attribute is removed in the final loom jupyter notebooks (see dir "7_downsampled_data_looms").